In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
transitions = pd.read_excel("data/transitions.xlsx")

In [ ]:
case_durations = transitions[['case:concept:name','@@flow_time_minute']].groupby('case:concept:name').sum()
case_stats = case_durations["@@flow_time_minute"].describe()[["min", "25%", "50%", "75%", "max"]]

In [ ]:
plt.figure(figsize=(6, 6))
sns.boxplot(y='@@flow_time_minute', data=case_durations)

# Label the plot
plt.xlabel('All Data', fontsize=14)
plt.ylabel('Time Spent', fontsize=14)
plt.title('Case Durations Stats in Minutes', fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define multiple flows to compare
selected_flows = ["Surgery Start > Surgery Stop"]

# Filter the DataFrame
df_filtered = transitions[transitions["transition"].isin(selected_flows)]

# Calculate summary statistics
stats = df_filtered.groupby("transition")["@@flow_time_minute"].describe()[["min", "25%", "50%", "75%", "max"]]
stats.rename(columns={"25%": "Q1", "50%": "Median", "75%": "Q3"}, inplace=True)

# Ensure the stats are sorted in the same order as the boxplot
ordered_transitions = df_filtered["transition"].unique()
stats = stats.loc[ordered_transitions]

# Transpose the DataFrame so rows become Min, Q1, Median
stats_t = stats.T

# Create figure and boxplot
fig, ax = plt.subplots(figsize=(10, 8))
ax.set(xlabel=None)

# Boxplot
sns.boxplot(x="transition", y="@@flow_time_minute", hue="transition", data=df_filtered, palette="Set2", ax=ax)

# Adjust x-axis labels for readability
plt.xticks([])
plt.xlabel("")
plt.ylabel("Flow Time (minutes)")
plt.title("Flow Time (minutes) Boxplot with Summary Statistics Table")

# Convert stats DataFrame to a table
table = plt.table(cellText=stats_t.values,  # Round values for readability
                  colLabels=stats_t.columns,        # Column headers
                  rowLabels=stats_t.index,          # Row headers
                  cellLoc='center',               # Center align text
                  loc='bottom')                   # Place table below plot

# Adjust layout so the table fits
plt.subplots_adjust(bottom=0.05)

# Show plot
plt.show()

In [ ]:
# Compute Flow Time Matrix (Mean)
flow_time_matrix = transitions.pivot_table(
    index="concept:name",
    columns="concept:name_2",
    values="@@flow_time_minute",
    aggfunc="mean"
).fillna(0)

# Plot Heatmap for Flow Time
plt.figure(figsize=(15, 6))
sns.heatmap(flow_time_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Flow Time Matrix (Mean Duration)")
plt.show()